<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Creating-a-Scorecard" data-toc-modified-id="Creating-a-Scorecard-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating a Scorecard</a></span><ul class="toc-item"><li><span><a href="#Scorecard-formulas" data-toc-modified-id="Scorecard-formulas-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Scorecard formulas</a></span></li></ul></li></ul></div>

# Import Libraries

In [1]:
import numpy as np
import pandas as pd

# Creating a Scorecard

In [2]:
df_scorecard = pd.read_csv('https://raw.githubusercontent.com/bhishanpdl/Shared/master/data/credit_risk_365_summary_table.csv',index_col=0)

df_scorecard

,Feature name,Coefficients,p_values
0,Intercept,-1.338448,NaN
1,grade:A,1.125487,2.457286e-35
2,grade:B,0.884349,9.337579e-48
3,grade:C,0.689487,2.091123e-34
4,grade:D,0.501113,5.861764e-21
...,...,...,...
80,mths_since_last_record:3-20,0.327600,5.235569e-03
81,mths_since_last_record:21-31,0.248542,1.990911e-02
82,mths_since_last_record:32-80,0.398012,2.795374e-07
83,mths_since_last_record:81-86,0.038353,6.562836e-01


In [3]:
df_scorecard['Original feature name'] = df_scorecard['Feature name'].str.split(':').str[0]

In [4]:
min_score = 300
max_score = 850

In [5]:
df_scorecard.groupby('Original feature name')['Coefficients'].min()

Original feature name
Intercept                     -1.338448
acc_now_delinq                 0.219342
addr_state                     0.038113
annual_inc                    -0.074434
dti                            0.047454
emp_length                     0.061224
grade                          0.135913
home_ownership                 0.093442
initial_list_status            0.054466
inq_last_6mths                 0.337905
int_rate                       0.112125
mths_since_earliest_cr_line    0.036304
mths_since_issue_d            -0.068385
mths_since_last_delinq         0.141652
mths_since_last_record         0.038353
purpose                        0.197680
term                           0.078726
verification_status           -0.010952
Name: Coefficients, dtype: float64

In [6]:
min_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].min().sum()
min_sum_coef

0.10048146420836424

In [7]:
df_scorecard.groupby('Original feature name')['Coefficients'].max()

Original feature name
Intercept                     -1.338448
acc_now_delinq                 0.219342
addr_state                     0.523819
annual_inc                     0.565699
dti                            0.381668
emp_length                     0.125832
grade                          1.125487
home_ownership                 0.106231
initial_list_status            0.054466
inq_last_6mths                 0.695516
int_rate                       0.883899
mths_since_earliest_cr_line    0.128538
mths_since_issue_d             1.084249
mths_since_last_delinq         0.213964
mths_since_last_record         0.398012
purpose                        0.301283
term                           0.078726
verification_status            0.084861
Name: Coefficients, dtype: float64

In [8]:
max_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].max().sum()
max_sum_coef

5.633142512038807

In [9]:
df_scorecard['Score - Calculation'] = df_scorecard['Coefficients'] * (max_score - min_score) / (max_sum_coef - min_sum_coef)
df_scorecard.head()

,Feature name,Coefficients,p_values,Original feature name,Score - Calculation
0,Intercept,-1.338448,NaN,Intercept,-133.054714
1,grade:A,1.125487,2.457286e-35,grade,111.884265
2,grade:B,0.884349,9.337579e-48,grade,87.912844
3,grade:C,0.689487,2.091123e-34,grade,68.541714
4,grade:D,0.501113,5.861764e-21,grade,49.815522


## Scorecard formulas

- https://i.imgur.com/Elhikpz.png

In [10]:
df_scorecard['Score - Calculation'][0] = ( 
    ( df_scorecard['Coefficients'][0]
     - min_sum_coef
    ) 
    / 
    (max_sum_coef - min_sum_coef)
    ) * (max_score - min_score) + min_score


df_scorecard.head()

/Users/poudel/miniconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Feature name,Coefficients,p_values,Original feature name,Score - Calculation
0,Intercept,-1.338448,NaN,Intercept,156.956456
1,grade:A,1.125487,2.457286e-35,grade,111.884265
2,grade:B,0.884349,9.337579e-48,grade,87.912844
3,grade:C,0.689487,2.091123e-34,grade,68.541714
4,grade:D,0.501113,5.861764e-21,grade,49.815522


In [11]:
df_scorecard['Score - Preliminary'] = df_scorecard['Score - Calculation'].round()
df_scorecard.head()

,Feature name,Coefficients,p_values,Original feature name,Score - Calculation,Score - Preliminary
0,Intercept,-1.338448,NaN,Intercept,156.956456,157.0
1,grade:A,1.125487,2.457286e-35,grade,111.884265,112.0
2,grade:B,0.884349,9.337579e-48,grade,87.912844,88.0
3,grade:C,0.689487,2.091123e-34,grade,68.541714,69.0
4,grade:D,0.501113,5.861764e-21,grade,49.815522,50.0


In [12]:
min_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Preliminary'].min().sum()
min_sum_score_prel

302.0

In [13]:
max_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Preliminary'].max(). sum()
max_sum_score_prel

851.0

In [14]:
df_scorecard['Difference'] = df_scorecard['Score - Preliminary'] - df_scorecard['Score - Calculation']

df_scorecard.head()

,Feature name,Coefficients,p_values,Original feature name,Score - Calculation,Score - Preliminary,Difference
0,Intercept,-1.338448,NaN,Intercept,156.956456,157.0,0.043544
1,grade:A,1.125487,2.457286e-35,grade,111.884265,112.0,0.115735
2,grade:B,0.884349,9.337579e-48,grade,87.912844,88.0,0.087156
3,grade:C,0.689487,2.091123e-34,grade,68.541714,69.0,0.458286
4,grade:D,0.501113,5.861764e-21,grade,49.815522,50.0,0.184478


In [15]:
max_diff_index = df_scorecard['Difference'].argmax()
max_diff_index

31

In [16]:
df_scorecard.sort_values('Difference',ascending=False).head()

,Feature name,Coefficients,p_values,Original feature name,Score - Calculation,Score - Preliminary,Difference
31,emp_length:2-4,0.125832,2.594486e-11,emp_length,12.508873,13.0,0.491127
6,grade:F,0.135913,5.591069e-03,grade,13.511085,14.0,0.488915
69,dti:7.7-10.5,0.316997,1.151056e-07,dti,31.512565,32.0,0.487435
3,grade:C,0.689487,2.091123e-34,grade,68.541714,69.0,0.458286
16,addr_state:GA_WA_OR,0.186669,2.542772e-12,addr_state,18.556657,19.0,0.443343


In [17]:
score_for_max_diff = df_scorecard.loc[max_diff_index,'Score - Preliminary']

score_for_max_diff

13.0

In [18]:
df_scorecard['Score - Final'] = df_scorecard['Score - Preliminary']
df_scorecard['Score - Final'][max_diff_index] = score_for_max_diff - 1
df_scorecard.head()

/Users/poudel/miniconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Feature name,Coefficients,p_values,Original feature name,Score - Calculation,Score - Preliminary,Difference,Score - Final
0,Intercept,-1.338448,NaN,Intercept,156.956456,157.0,0.043544,157.0
1,grade:A,1.125487,2.457286e-35,grade,111.884265,112.0,0.115735,112.0
2,grade:B,0.884349,9.337579e-48,grade,87.912844,88.0,0.087156,88.0
3,grade:C,0.689487,2.091123e-34,grade,68.541714,69.0,0.458286,69.0
4,grade:D,0.501113,5.861764e-21,grade,49.815522,50.0,0.184478,50.0


In [19]:
min_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Final'].min().sum()
min_sum_score_prel

302.0

In [20]:
max_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Final'].max(). sum()
max_sum_score_prel

850.0

In [21]:
df_scorecard

,Feature name,Coefficients,p_values,Original feature name,Score - Calculation,Score - Preliminary,Difference,Score - Final
0,Intercept,-1.338448,NaN,Intercept,156.956456,157.0,0.043544,157.0
1,grade:A,1.125487,2.457286e-35,grade,111.884265,112.0,0.115735,112.0
2,grade:B,0.884349,9.337579e-48,grade,87.912844,88.0,0.087156,88.0
3,grade:C,0.689487,2.091123e-34,grade,68.541714,69.0,0.458286,69.0
4,grade:D,0.501113,5.861764e-21,grade,49.815522,50.0,0.184478,50.0
...,...,...,...,...,...,...,...,...
80,mths_since_last_record:3-20,0.327600,5.235569e-03,mths_since_last_record,32.566571,33.0,0.433429,33.0
81,mths_since_last_record:21-31,0.248542,1.990911e-02,mths_since_last_record,24.707481,25.0,0.292519,25.0
82,mths_since_last_record:32-80,0.398012,2.795374e-07,mths_since_last_record,39.566276,40.0,0.433724,40.0
83,mths_since_last_record:81-86,0.038353,6.562836e-01,mths_since_last_record,3.812685,4.0,0.187315,4.0


In [22]:
df_scorecard['Score - Final'].describe()

count     85.000000
mean      28.482353
std       28.783102
min       -7.000000
25%        9.000000
50%       21.000000
75%       37.000000
max      157.000000
Name: Score - Final, dtype: float64